In [ ]:
import math
import matplotlib.pyplot as plt

In [ ]:
def find_lr(model, loss_fn, optimizer, train_loader, init_value=1e-8, final_value=10.0):
    
    number_in_epoch = len(train_loader) - 1
    
    update_step = (final_value / init_value) ** (1 / number_in_epoch)
    
    lr = init_value
    
    optimizer.param_groups[0]["lr"] = lr
    
    best_loss = float('inf')
    
    batch_num = 0
    
    losses = []
    log_lrs = []
    
    for data in train_loader:
        batch_num += 1 
        
        inputs, labels = data

        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        loss = loss_fn(outputs, labels)
        
        if batch_num > 1 and loss > 4 * best_loss:
            return log_lrs[10:-5], losses[10:-5]
        
        if loss < best_loss or batch_num == 1:
            best_loss = loss
        
        losses.append(loss.item())
        log_lrs.append(math.log10(lr))
        
        loss.backward()
        
        optimizer.step()
        
        lr *= update_step
        optimizer.param_groups[0]["lr"] = lr
    
    return log_lrs[10:-5], losses[10:-5]

In [ ]:
logs, losses = find_lr(model, loss_fn, optimizer, train_loader)

In [ ]:
plt.plot(logs, losses)
plt.xlabel('Learning Rate (log scale)')
plt.ylabel('Loss')
plt.show()

In [ ]:
optimizer = optim.Adam([
    { 'params': transfer_model.layer4.parameters(), 'lr': found_lr / 3 },
    { 'params': transfer_model.layer3.parameters(), 'lr': found_lr / 9 }
], lr=found_lr)

In [ ]:
unfreeze_layers = [transfer_model.layer3, transfer_model.layer4]
for layer in unfreeze_layers:
    for param in layer.parameters():
        param.requires_grad = True